In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 64
CHANNELS=3
EPOCHS=25

In [3]:
dataset = tf.keras.utils.image_dataset_from_directory(
    "DiseaseDataset",
    shuffle = True,
    seed=123,
    image_size = (294,222),
    batch_size = BATCH_SIZE
)

Found 10000 files belonging to 10 classes.


In [4]:
class_names = dataset.class_names
class_names

['1. Eczema',
 '10. Warts Molluscum',
 '2. Melanoma',
 '3. Atopic Dermatitis',
 '4. Basal Cell Carcinoma',
 '5. Melanocytic Nevi',
 '6. Benign Keratosis',
 '7. Psoriasis pictures Lichen Planus',
 '8. Seborrheic Keratoses',
 '9. Tinea Ringworm Candidiasis']

In [5]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch[0].shape)

(294, 222, 3)


In [6]:
len(dataset)

157

In [7]:
train_size = 0.8
len(dataset)*train_size

125.60000000000001

In [9]:
train_ds=dataset.take(125)
len(train_ds)

125

In [10]:
test_ds = dataset.skip(125)
len(test_ds)

32

In [11]:
val_ds = test_ds.take(16)
test_ds = test_ds.skip(16)

In [12]:
len(test_ds)

16

In [13]:
len(val_ds)

16

In [14]:
len(train_ds)

125

In [15]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    ds_size=len(ds)

    if shuffle:
        ds=ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds
    

In [16]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [17]:
len(train_ds)

125

In [18]:
len(val_ds)

15

In [19]:
len(test_ds)

17

In [20]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [21]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(294, 222),
    layers.experimental.preprocessing.Rescaling(1./255)
])

In [22]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [22]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [23]:
input_shape = (BATCH_SIZE, 294, 222, CHANNELS)

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(294,222,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(294,222,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(294,222,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(294,222,3)),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax'),    
])

model.build(input_shape=input_shape)

In [24]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [25]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=5
)

Epoch 1/5
125/125 [==============================] - 500s 4s/step - loss: 1.9723 - accuracy: 0.2529 - val_loss: 1.7176 - val_accuracy: 0.3354
Epoch 2/5
125/125 [==============================] - 424s 3s/step - loss: 1.6591 - accuracy: 0.3701 - val_loss: 1.5897 - val_accuracy: 0.4000
Epoch 3/5
125/125 [==============================] - 418s 3s/step - loss: 1.5669 - accuracy: 0.3956 - val_loss: 1.5065 - val_accuracy: 0.4271
Epoch 4/5
125/125 [==============================] - 419s 3s/step - loss: 1.5012 - accuracy: 0.4284 - val_loss: 1.4117 - val_accuracy: 0.4729
Epoch 5/5
125/125 [==============================] - 415s 3s/step - loss: 1.4445 - accuracy: 0.4432 - val_loss: 1.4934 - val_accuracy: 0.4104


In [26]:
scores = model.evaluate(test_ds)

17/17 [==============================] - 43s 646ms/step - loss: 1.3974 - accuracy: 0.4721
